In [105]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [118]:
#print(os.environ["CUDA_VISIBLE_DEVICES"])

KeyError: 'CUDA_VISIBLE_DEVICES'

In [120]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [121]:
print(check_output(["ls" , "../input"]).decode("utf8"))

glove.6B.100d.txt
glove.6B.200d.txt
glove.6B.300d.txt
glove.6B.50d.txt
glove.6B.zip
glove.840B.300d.txt
glove.840B.300d.zip
sample_submission.csv
test.csv
train.csv



In [122]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

#train=train.sample(frac=1)

In [123]:
train_sentences=train.comment_text.fillna("_na_").values
test_sentences=test.comment_text.fillna("_na_").values
class_cloumns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y= train[class_cloumns].values

In [124]:
train_sentences[0:5]

array([ "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences 

In [125]:
# Basic configuration parameters
embed_size = 50
max_features = 20000
maxlen = 100



#Create a Tokenizer class objectand then fit the tokenizer to the training set
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sentences))
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
tokenized_test=tokenizer.texts_to_sequences(test_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)
X_train[0:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   688,
           75,     1,   126,   130,   177,    29,   672,  4511, 12052,
         1116,    86,   331,    51,  2278, 11448,    50,  6864,    15,
           60,  2756,   148,     7,  2937,    34,   117,  1221, 15190,
         2825,     4,    45,    59,   244,     1,   365,    31,     1,
           38,    27,   143,    73,  3462,    89,  3085,  4583,  2273,
          985],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [126]:
embedding_file='../input/glove.6B.50d.txt'
#embeddings_index = pd.read_table(embedding_file, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)


In [127]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file))

In [128]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940488, 0.64410424)

In [129]:
# Word index dictionary of tokenized words
import random
random.seed(4829975900407156849)
word_index=tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)
#print(embedding_matrix[0:2])
print(emb_mean, emb_std, (nb_words, embed_size))
print(X_train.shape)
print(X_train[0:70,10])
print(embedding_matrix[0:5])

(20000, 50)
0.0209405 0.644104 (20000, 50)
(159571, 100)
[   0    0    0    3    0    0    0    0    0    0  391    0    0 1128    0
  798    0    0    0    0    0    0    0    0 1309    0    0    0    0    0
    0    0    0  508    0  225    5    0    0  301    0    0   29    0    0
 1011    0    0   18   46    1    0    0    0    0    0    0 1310    7    0
    0    0    0 1966    0    0    0    0    0    0]
[[ -2.46331076e-01   9.85731994e-02  -2.09594828e-01   4.28027846e-01
    2.99032550e-01   7.86187026e-01  -5.90618598e-01   1.07381302e+00
   -3.73605614e-01   1.07821286e+00   1.75973765e-01   6.29861359e-01
   -5.38836693e-01   2.93602594e-01  -3.30034771e-01  -4.77305239e-02
   -3.69043331e-02  -6.22412120e-01   7.22511035e-01  -3.60349541e-01
    9.32559829e-02  -1.57864883e+00   5.87879482e-01   1.34319584e+00
   -3.81112670e-01  -5.83231530e-01  -2.10476059e-01  -5.41824654e-01
   -3.13815893e-01  -5.20551072e-01  -7.44250395e-02   2.01783127e-02
    1.88962174e-01  -1.0719

In [130]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = Bidirectional(LSTM(50, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = BatchNormalization()(x)
x = Dense(50, activation="relu")(x)
#x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

import keras.backend as K
def loss(y_true, y_pred):
     return K.binary_crossentropy(y_true, y_pred)
    
model.compile(loss=loss, optimizer='nadam', metrics=['accuracy'])

In [131]:
def schedule(ind):
    a = [0.002,0.003, 0.000]
    return a[ind]
lr = callbacks.LearningRateScheduler(schedule)
model.fit(X_train, y, batch_size=64, epochs=3, validation_split=0.1, callbacks=[lr])

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
 49408/143613 [=========>....................] - ETA: 239s - loss: 0.6165 - acc: 0.3846

KeyboardInterrupt: 

In [ ]:
y_test = model.predict([X_test], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission[class_cloumns] = y_test
sample_submission.to_csv('LSTM-submission.csv', index=False)